# Analys av Travdata

Denna notebook analyserar det skapade datasetet `data/processed/race_data.csv`.
Fokus ligger på att förstå datakvaliteten och verifiera tidsuppdelningen för träning/test.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Inställningar
pd.set_option('display.max_columns', 50)
sns.set_theme(style="whitegrid")

In [ ]:
# Ladda data
df = pd.read_csv('../data/processed/race_data.csv')

# Konvertera datum
df['date'] = pd.to_datetime(df['date'])

print(f"Totalt antal rader: {len(df)}")
print(f"Tidsperiod: {df['date'].min()} till {df['date'].max()}")
df.head()

## Tidsuppdelning (Train/Test Split)

Vi ska använda data **innan 2026-01-05** för träning.
Data från och med **2026-01-05** behålls som holdout (test) för att simulera verkligheten.

In [ ]:
split_date = '2026-01-05'

train_df = df[df['date'] < split_date]
test_df = df[df['date'] >= split_date]

print(f"Träningsdata (före {split_date}): {len(train_df)} starter ({len(train_df)/len(df):.1%})")
print(f"Testdata (från {split_date}): {len(test_df)} starter ({len(test_df)/len(df):.1%})")

## Analys av Odds-data (30min / 5min innan start)

Vi vill se hur många lopp som faktiskt har oddsdata från tidpunkterna innan start (detta finns främst i nyare data).

In [ ]:
print("Saknade värden i Träningsdata:")
print(train_df[['odds_30m', 'odds_5m']].isna().mean())

print("\nSaknade värden i Testdata:")
print(test_df[['odds_30m', 'odds_5m']].isna().mean())

# Plot av tillgänglighet över tid
df['has_odds_5m'] = df['odds_5m'].notna()
daily_coverage = df.groupby('date')['has_odds_5m'].mean().reset_index()

plt.figure(figsize=(12, 4))
sns.lineplot(data=daily_coverage, x='date', y='has_odds_5m')
plt.axvline(pd.to_datetime(split_date), color='r', linestyle='--', label='Split Date')
plt.title('Andel starter med odds-data (5 min innan) per dag')
plt.legend()
plt.show()

## Target Distribution (Målvariabel)
Vad försöker vi förutsäga? Ofta vill vi hitta vinnaren (`finish_order == 1`).

In [ ]:
# Kolla fördelning av finish_order
sns.countplot(x=train_df[train_df['finish_order'] <= 12]['finish_order'])
plt.title('Målplacering i träningsdata')